In [338]:
import numpy as np
from sklearn.model_selection import train_test_split

In [339]:
encoded_data=np.loadtxt('./encoded_input_noise/encodedInputNoise_2_1_2.txt',delimiter=',')
msg_data=np.loadtxt('./msg/msg_2_1_2.txt',delimiter=',')


In [340]:
encoded_data_train,encoded_data_test,msg_data_train,msg_data_test=train_test_split(encoded_data,msg_data,test_size=0.33,random_state=42)

In [341]:
msg_data_bpsk_train=1-2*msg_data_train
msg_data_bpsk_test=1-2*msg_data_test

In [342]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [343]:
data_in=keras.Input(shape=(136,))

In [344]:
x_dense=layers.Dense(70,activation='tanh')(data_in)
data_out=layers.Dense(44,activation='tanh')(x_dense)

In [345]:
model=keras.Model(inputs=data_in,outputs=data_out)

In [346]:
model.compile(optimizer="adam",loss='hinge')

In [347]:
print(encoded_data_train.shape,":",msg_data_train.shape)

(40200, 136) : (40200, 44)


In [348]:
history=model.fit(encoded_data_train,msg_data_bpsk_train,epochs=10)

Epoch 1/10
1257/1257 [==============================] - 3s 2ms/step - loss: 0.4460
Epoch 2/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.2577
Epoch 3/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.2341
Epoch 4/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.2212
Epoch 5/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.2127
Epoch 6/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.2062
Epoch 7/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.2016
Epoch 8/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.1977
Epoch 9/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.1948
Epoch 10/10
1257/1257 [==============================] - 2s 1ms/step - loss: 0.1924


In [349]:
print(history.history)

{'loss': [0.44602397084236145, 0.25772780179977417, 0.2340850681066513, 0.22116325795650482, 0.2127111256122589, 0.20620034635066986, 0.20159225165843964, 0.19769258797168732, 0.19479334354400635, 0.19238166511058807]}


In [350]:
model.evaluate(encoded_data_test,msg_data_bpsk_test,batch_size=100)

198/198 [==============================] - 0s 1ms/step - loss: 0.1978


0.19781775772571564

In [358]:
predicted=model.predict(encoded_data_test)

In [359]:
predicted

array([[ 0.99999577,  0.8059409 ,  0.99954224, ...,  1.        ,
         0.99883896, -0.99750334],
       [ 0.99942833, -0.39342064, -0.9863648 , ...,  1.        ,
         0.9999697 , -0.9999829 ],
       [ 0.9999938 ,  0.8476758 , -0.9989404 , ...,  0.99989015,
        -1.        , -1.        ],
       ...,
       [ 0.06045552, -0.9710391 , -0.9973756 , ...,  0.2975515 ,
        -0.99998546,  0.99967885],
       [ 1.        ,  0.9999997 , -0.9999434 , ..., -0.99676317,
         0.8652765 ,  0.65484655],
       [-0.73541266,  0.9975856 ,  0.9999943 , ...,  0.9044324 ,
        -0.99975836,  0.9950652 ]], dtype=float32)

In [360]:
predicted=np.where(predicted<0,-1,1)
            









In [361]:
predicted

array([[ 1,  1,  1, ...,  1,  1, -1],
       [ 1, -1, -1, ...,  1,  1, -1],
       [ 1,  1, -1, ...,  1, -1, -1],
       ...,
       [ 1, -1, -1, ...,  1, -1,  1],
       [ 1,  1, -1, ..., -1,  1,  1],
       [-1,  1,  1, ...,  1, -1,  1]])

In [362]:
msg_data_bpsk_test

array([[ 1.,  1.,  1., ...,  1.,  1., -1.],
       [ 1., -1., -1., ...,  1.,  1., -1.],
       [ 1.,  1., -1., ...,  1., -1., -1.],
       ...,
       [ 1., -1., -1., ..., -1., -1.,  1.],
       [ 1.,  1., -1., ..., -1.,  1., -1.],
       [-1.,  1.,  1., ...,  1., -1.,  1.]])

In [356]:
error=0
for i in range(0,len(predicted)):
    for j in range(0,len(predicted[i])):
        if(predicted[i][j]!=msg_data_bpsk_test[i][j]):
            error+=1
total_bits=len(predicted)*len(predicted[0])
ber=error/total_bits

In [357]:
ber

0.08910812672176309

In [405]:
def data_prep():
    encoded_data=np.loadtxt('./encoded_input_noise/encodedInputNoise_1_1_2.txt',delimiter=',')
    msg_data=np.loadtxt('./msg/msg_1_1_2.txt',delimiter=',')
    encoded_data_train,encoded_data_test,msg_data_train,msg_data_test=train_test_split(encoded_data,msg_data,test_size=0.33,random_state=42)
    msg_data_bpsk_train=1-2*msg_data_train
    msg_data_bpsk_test=1-2*msg_data_test
    return [encoded_data_train,encoded_data_test,msg_data_bpsk_train,msg_data_bpsk_test]
    
def model():
    data_in=keras.Input(shape=(136,))
    x_dense=layers.Dense(70,activation='tanh')(data_in)
    data_out=layers.Dense(44,activation='tanh')(x_dense)
    Neural_Network=keras.Model(inputs=data_in,outputs=data_out)
    Neural_Network.compile(optimizer='adam',loss='hinge')
    
    data=data_prep()
    training_data=data[0]
    training_data_labels=data[2]
    trained_nn=Neural_Network.fit(training_data,training_data_labels,epochs=10)
    
    #predictions
    test_data=data[1]
    test_data_labels=data[3]

    predictions=Neural_Network.predict(test_data)
    predictions=np.where(predictions<0,-1,1)

    true=sum(np.ravel(predictions==test_data_labels))
    
    total_bits=len(predictions)*len(predictions[0])

    Ber=(total_bits-true)/total_bits
    print(total_bits,":",total_bits-true)
    return Ber

    
    
    
print(model())
    


Epoch 1/10
1257/1257 [==============================] - 1s 797us/step - loss: 0.4846
Epoch 2/10
1257/1257 [==============================] - 1s 803us/step - loss: 0.3035
Epoch 3/10
1257/1257 [==============================] - 1s 780us/step - loss: 0.2809
Epoch 4/10
1257/1257 [==============================] - 1s 786us/step - loss: 0.2689
Epoch 5/10
1257/1257 [==============================] - 1s 787us/step - loss: 0.2607
Epoch 6/10
1257/1257 [==============================] - 1s 780us/step - loss: 0.2546
Epoch 7/10
1257/1257 [==============================] - 1s 819us/step - loss: 0.2500
Epoch 8/10
1257/1257 [==============================] - 1s 784us/step - loss: 0.2463
Epoch 9/10
1257/1257 [==============================] - 1s 776us/step - loss: 0.2434
Epoch 10/10
1257/1257 [==============================] - 1s 777us/step - loss: 0.2410
871200 : 99217
0.1138854453627181


In [388]:
a=[[1,2,3],[4,5,6]]
b=[[1,5,6],[9,4,6]]
a=np.asmatrix(a)
b=np.asmatrix(b)
c=a==b


2

In [375]:
a

matrix([[1, 2, 3],
        [4, 5, 6]])

In [368]:
b=np.asarray([1,3,5,6])
sum(a==b)

1